In [1]:
from prevo.viewers import TkWindow, TkViewer
from prevo.viewers import CvWindow, CvViewer
from prevo.viewers import MplWindow, MplViewer

from prevo.misc import PeriodicSensor
from threading import Thread, Event
import oclock
import numpy as np
%matplotlib tk

# 1) MISC tools

## Dummy camera sensor

The section below is just to define a dummy class that mimicks a camera sending images on a queue.

In [2]:
class LapseCamera(PeriodicSensor):
    """Mock time-lapse camera returning white-noise images periodically"""
    
    name = 'Mock Lapse Camera'
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num = 0
    
    def _read(self):
        """Return image in a dict (see explanation below)"""
        img = np.random.randint(256, size=(480, 640), dtype='uint8')
        data = {'image': img, 'num': self.num}
        self.num += 1
        return data

In [3]:
camera1 = LapseCamera(interval=0.04)
camera1.start()

camera2 = LapseCamera(interval=2)
camera2.start()

camera3 = LapseCamera(interval=0.5)
camera3.start()

## External command to stop display

In [4]:
def stop_display(after=10):
    """By default, send stop request after 10 seconds"""
    stop_event = Event()
    
    def _stop_display():
        timer = oclock.Timer()
        while timer.elapsed_time < 10:
            timer.checkpt()
        print('STOP REQUESTED')
        stop_event.set()
        
    Thread(target=_stop_display).start()
    return stop_event

# 2) Viewers operation

Viewers operate independent windows (one per image source). Each window accepts a queue (`queue.Queue` objects or equivalent) as input. By default, it is assumed that the objects in the queue are dictionaries with a key `'image'` containing the image (`numpy` array or equivalent). If necessary, this behavior can be changed by changing the window's `measurement_formatter` (see example further below in **3) Subclassing**).

**NOTE**: If one wants to use the `show_num` option, the measurement dictionary must also contain a `'num'` key, with the image number as a value. Here it's also possible to customize with the user-supplied `measurement_formatter`.

**NOTE**: the FPS options in the arguments refer to the fps displayed in the window, not the actual FPS of the camera:
- `calculate_fps=True` creates a `display_times` attribute (list) of the viewer where times are stored, and prints the average FPS when viewer is closed
- `show_fps=True` shows the current (live) display FPS in the window

## Tkinter

In [5]:
win1 = TkWindow(camera1.queue, name='Camera 1', show_fps=True)
win2 = TkWindow(camera2.queue, name='Camera 2', show_num=True)
TkViewer(windows=(win1, win2)).start()

## OpenCV

In [6]:
win1 = CvWindow(camera1.queue, name='Camera 1', show_fps=True)
win2 = CvWindow(camera2.queue, name='Camera 2', show_num=True)

# Here we include an external signal after 10 seconds to stop the viewer
viewer = CvViewer(
    windows=(win1, win2),
    external_stop=stop_display(after=10),
)
viewer.start()

STOP REQUESTED


## Matplotlib

**NOTE**: For some reason on some platforms the Matplotlib FuncAnimation does not stop even when closed in a Jupyter environment, so the kernel has to be restarted at the end.

**NOTE**: In some cases, the exiting of the viewer also creates a bug in a `on_timer` thread. I have not been able to solve this at the moment, but it does not seem to crash the program in any case.

In [7]:
win1 = MplWindow(camera1.queue, name='Camera 1', show_fps=True)
win2 = MplWindow(camera2.queue, name='Camera 2', show_num=True)
win3 = MplWindow(camera3.queue, name='Camera 3', calculate_fps=True)

viewer = MplViewer(
    windows=(win1, win2, win3),
    blit=False,
    external_stop=stop_display(after=10),
)
viewer.start()

STOP REQUESTED
Average display frame rate [Camera 3]: 2.041 fps. 
Average display frame rate [Camera 3]: 2.041 fps. 


invalid command name "5184348416_on_timer"
    while executing
"5184348416_on_timer"
    ("after" script)


# 3) Subclassing

The most obvious case for subclassing is when the format of measurements stored in the queues is not that by default (dict with key `image` and `num`). Below is an example of providing another `measurement_formatter` to the windows in a case where the camera queue contains only the image array.

We also show how to subclass the Tkinter window class in order to not have to pass the new `measurement_formatter` every time.

## Dummy sensor

In [8]:
class ModifiedLapseCamera(PeriodicSensor):
    """Mock camera sensor that returns images directly instead of dicts."""
    def _read(self):
        img = np.random.randint(256, size=(480, 640), dtype='uint8')
        return img

In [9]:
simple_cam_1 = ModifiedLapseCamera(interval=0.1)
simple_cam_1.start()

simple_cam_2 = ModifiedLapseCamera(interval=0.5)
simple_cam_2.start()

## Custom measurement formatter

In [10]:
class CustomFormatter:
    
    def __init__(self):
        """Since the camera source does not provide image number, we'll add it to the data with the formatter"""
        self.image_count = 0
        
    def get_image(self, measurement):
        """Each time an image is retrieved, we also update the image number.
        
        Note that here the first image will have a number of 1, not 0"""
        self.image_count += 1
        return measurement
    
    def get_num(self, measurement):
        return self.image_count

## Subclassing Tkinter window

In [11]:
class Window(TkWindow):
    
    def __init__(self, *args, **kwargs):
        custom_formatter = CustomFormatter()
        super().__init__(*args, measurement_formatter=custom_formatter, **kwargs)

## Running the viewer

In [12]:
win1 = Window(simple_cam_1.queue, name='Camera 1', show_fps=True)
win2 = Window(simple_cam_2.queue, name='Camera 2', show_num=True)
TkViewer((win1, win2)).start()